# FM算法实现（向量化）

In [1]:
import numpy as np
import pandas as pd
import random

random.seed(0)
np.random.seed(0)

trainData = './data/diabetes_train.txt'   #请换为自己文件的路径
testData = './data/diabetes_test.txt'

def preprocessData(data):
    feature=np.array(data.iloc[:,:-1])   #取特征
    label=data.iloc[:,-1].map(lambda x: 1 if x==1 else -1) #取标签并转化为 +1，-1
    #将数组按行进行归一化
    zmax, zmin = feature.max(axis=0), feature.min(axis=0)
    feature = (feature - zmin) / (zmax - zmin)
    label=np.array(label)
    return feature,label

def sigmoid(data):
    return 1.0 / (1+np.exp(-data))

class FM:
    def __init__(self, k=8, learning_rate = 0.005):
        # 隐向量维数
        self.k = k
        # 学习率
        self.learning_rate = learning_rate

    def fit(self, data, label, iter):
        m, n = data.shape
        # 常数项系数
        self.w0 = 0.0
        #  一次项系数
        # self.w = np.random.normal(size=(n,1))
        self.w = np.zeros((n,1))
        # 二次项系数矩阵，即FM核心思想的隐向量系数矩阵
        # self.v = np.random.normal(size=(n,self.k))
        self.v = random.normalvariate(0, 0.2) * np.ones((n, self.k))

        for epoch in range(iter):
            # 采用SGD算法求解
            for index in range(m):
                feature = data[index].reshape(-1,1)
                inter_1 = np.sum(feature * self.v, axis=0,keepdims=True)
                inter_2 = np.sum((feature * feature) * (self.v * self.v),axis=0,keepdims=True)
                interaction = np.sum(inter_1 * inter_1 - inter_2, axis=1) / 2.0
                # 预测值
                pred = self.w0 + np.dot(feature.T,self.w) + interaction[0]
                # loss 部分梯度，即loss对sigmoid求导的结果方便后续计算
                loss = 1 - sigmoid(label[index] * pred[0,0])

                # 分别更新参数
                self.w0 = self.w0 + self.learning_rate * loss * label[index]
                self.w = self.w + self.learning_rate * loss  * label[index] * feature
                self.v = self.v + self.learning_rate * loss * label[index] * (feature * inter_1 -
                                                               (feature * feature) * self.v)
            print("第{}次迭代后的loss部分梯度为{}".format(epoch, loss))

    def predict(self,data):
        inter_1 = data.dot(self.v)
        inter_2 = (data * data).dot(self.v * self.v)
        interaction = np.sum(inter_1 * inter_1 - inter_2, axis=1) / 2.0

        # 预测值
        pred = self.w0 +data.dot(self.w) + interaction.reshape(-1,1)
        pred = np.where(pred>0.5, 1, -1)
        return pred

if __name__ == "__main__":
    train=pd.read_csv(trainData)
    test = pd.read_csv(testData)
    dataTrain, labelTrain = preprocessData(train)
    dataTest, labelTest = preprocessData(test)

    clf = FM(k=20)
    clf.fit(dataTrain, labelTrain,200)
    res = clf.predict(dataTest)
    prec = res == labelTest.reshape(-1,1)
    print("准确率：", np.sum(prec)/dataTest.shape[0])

第0次迭代后的loss部分梯度为0.41668464877653133
第1次迭代后的loss部分梯度为0.3947650728939792
第2次迭代后的loss部分梯度为0.39614143824248094
第3次迭代后的loss部分梯度为0.4036920221903222
第4次迭代后的loss部分梯度为0.41314481820913174
第5次迭代后的loss部分梯度为0.42331589134482717
第6次迭代后的loss部分梯度为0.4337703555918372
第7次迭代后的loss部分梯度为0.4442493542185031
第8次迭代后的loss部分梯度为0.4545474290025685
第9次迭代后的loss部分梯度为0.46449822641563443
第10次迭代后的loss部分梯度为0.47397763260101977
第11次迭代后的loss部分梯度为0.4829031863916269
第12次迭代后的loss部分梯度为0.49122836822549876
第13次迭代后的loss部分梯度为0.49893466334881764
第14次迭代后的loss部分梯度为0.5060238330321158
第15次迭代后的loss部分梯度为0.5125115135247461
第16次迭代后的loss部分梯度为0.5184223447910306
第17次迭代后的loss部分梯度为0.5237864247634645
第18次迭代后的loss部分梯度为0.5286367883078127
第19次迭代后的loss部分梯度为0.5330076412918514
第20次迭代后的loss部分梯度为0.5369331423079942
第21次迭代后的loss部分梯度为0.5404465810361743
第22次迭代后的loss部分梯度为0.5435798446183919
第23次迭代后的loss部分梯度为0.5463630932654394
第24次迭代后的loss部分梯度为0.5488245871285184
第25次迭代后的loss部分梯度为0.5509906212267897
第26次迭代后的loss部分梯度为0.5528855359726854
第27次迭代后的loss部分梯度为0.55453177887